In [ ]:
# Import needed libraries
import pandas as pd # To handle the data in a dataframe
import numpy as np # To calculate quartiles etc.
import plotly.graph_objects as go # To create the box plot.

## Upload and read in the data
First of all, we need to upload the data to make it available here i colab, which can be done in the following way:
1. Click the folder icon in the left side of the browser window, which will open the file view in the side bar.
2. Click the left-most icon (page with arrow pointing up), which will open a window to select a file from your local computer.
3. Select the file called data.xlsx, which has been distributed with this Colab notebook (you first need to download it to your computer).
4. The file will now appear in the side bar and we can use it in the code below.


In [ ]:
# Read the uploaded excel file into a pandas dataframe and put it into an object called df
df = pd.read_excel('data.xlsx')

# Show the content of the dataframe
df

## Create the box plot
To create a box plot in Python requires a choice between different methods to calculate the quartiles (also called 25- and 75-percentiles). Such a choice is not available in excel where the method is pre-defined.

According to [Journal of Statistics Education, Volume 14, Number 3 (2006)](https://jse.amstat.org/v14n3/langford.html) there are 15 different methods for calculating the quartiles/percentiles.

In the following, we create the boxplot with the Plotly library, which has three different methods for quartile calculations. The method used in the below code is selected to reproduce the plot and values from excel.

In [ ]:
# Create a figure object:
fig1 = go.Figure()

# Create a list with column names
col_names = list(df.columns)

# Loop over the columns and add each of them to the figure as box plots showing
# the points of measurements in the center of the box and the mean as a dotted line
for name in col_names:
  fig1.add_trace(go.Box(y=df[name], boxpoints='all', pointpos=0, boxmean=True,
                        name=name, quartilemethod='exclusive'))

# Format the figure size and provide plot and y-axis titles:
fig1.update_layout(title='Penge brugt til fredagsbar', width=1200, height=500,
                   yaxis_title='DKK')

# Display the figure:
fig1.show()


One of the big advantages of using Plotly to create plots/graphs is that you can place your mouse curser over the plot to display the values presented in each subplot. This applies to both the individual data points and the calculated values.

Additionally, placing your mouse curser over the plot also presents a graphical menu in the upper rigth corner from which, you can e.g. export your plot as a pgn-file (the camera icon), zoom and more.

## Calculate and export the values
As for Plotly above, to calculate quartiles (or 25-, 50- and 75-percentiles) in Python requires a choice of method and in Numpy used below there are 13 different methods.

The methods used in the below code are selected to reproduce the values from excel.

### Create a dataframe
First we create a new dataframe to contain the calculated values. We define the row and column names while creating the dataframe and use the same column names as in the original dataframe (from the "col_names" list defined above).

With respect to the row names, we have to make sure that the calculated values are added in the same order and end up in the correct rows.

In [ ]:
# Create the new dataframe with defined row and column names (index and columns)
# but no values
df2 = pd.DataFrame(index=['25% kvartil', 'Median', '75% kvartil',
                          'Inter kvartil afstand', 'Maks. grænse',
                          'Min. grænse', 'Største obs. indenfor grænsen',
                          'Mindste obs. indenfor grænsen', 'Middelværdi'], columns=col_names)

# Show the dataframe
df2

### Calculating the values
We want to make the code more efficient and not having to do the same procedure for each of the columns in the original dataframe. Thus, we make a loop:

In [ ]:
# Create a loop to calculate all the needed values for each column.
for name in col_names:
  # Create a list of values in the column
  val_list = df[name].tolist()
  # Create empty list to store calculated parameters
  results = []
  # Calculate q1 of val_list
  q1 = np.percentile(val_list, 25, method='lower')
  # Calculate the median of val_list
  medi = np.percentile(val_list, 50, method='lower')
  # Calculate q3 of val_list
  q3 = np.percentile(val_list, 75, method='higher')
  # Calculate iqr of val_list
  iqr = q3 - q1
  # Calculate the upper limit for outliers
  up_limit = q3 + 1.5 * iqr
  # Calculate the lower limit for outliers
  low_limit = q1 - 1.5 * iqr
  # Create empty list to store values between limits
  within_limits = []
  # Loop over values in val_list
  for val in val_list:
    # Append the values between min and max to the "within_iqr" list
    if low_limit < val < up_limit:
      within_limits.append(val)
  # Get the maximum value from the list
  max_in_limits = max(within_limits)
  # Get the minimum value from the list
  min_in_limits = min(within_limits)
  # Calculate the mean of val_list
  mean = np.mean(val_list)
  # Extend the "results" list with the calculated parameters rounded to 0 decimals and converted to integers
  # Please note that Python rounds "half to even" and does not produce exactly the same as excel (744,5 rounds to 744)
  results.extend(np.round([q1, medi, q3, iqr, up_limit, low_limit, max_in_limits, min_in_limits, mean],0).astype(int))
  # Print the "results"
  print('Værdier for', name, results)
  # Put the "results" list into the dataframe
  df2[name] = results

# Show the dataframe with all the calculated parameters
df2

### Export your results
In the below code cell we export the dataframe with our calculated parameters to an excel file. This will create a file in the "files side bar" where you previously uploaded the data file (you may have to update/refresh the view by clicking the second icon - folder with circular arrow).

In [ ]:
# Export the dataframe with calculated parameter to an excel file
df2.to_excel('results.xlsx')

You can now download the generated excel file to your computer by "right-clicking" the file in the side bar and select "download".